<a href="https://colab.research.google.com/github/wiroel/my-repo/blob/main/final_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
import joblib

drive.mount('/content/drive')

final_rf = joblib.load('/content/drive/MyDrive/final_rf.pkl')
final_xg = joblib.load('/content/drive/MyDrive/final_xg.pkl')

X_train_resampled = joblib.load('/content/drive/MyDrive/X_train_resampled.pkl')
y_train_resampled = joblib.load('/content/drive/MyDrive/y_train_resampled.pkl')
X_test = joblib.load('/content/drive/MyDrive/X_test.pkl')
y_test = joblib.load('/content/drive/MyDrive/y_test.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import precision_recall_curve


from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix

# Hyperparameter tuning on random forest


In [15]:
final_rf.fit(X_train_resampled, y_train_resampled)


# this is for identifying the threshold for y_prob as the default is 0.5, this gives perffect threshold

from sklearn.metrics import precision_recall_curve

y_probs= final_rf.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, y_probs)


f1_score = 2 * (precision * recall) / (precision+recall)
best_threshold_rf = thresholds[f1_score.argmax()]

print(f"Best threshold: {best_threshold_rf}")



#y_pred = final_rf.predict(X_test)

y_pred = (y_probs >= best_threshold_rf).astype(int)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Best threshold: 0.1909101027698589
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1254
           1       0.37      0.80      0.50        66

    accuracy                           0.92      1320
   macro avg       0.68      0.86      0.73      1320
weighted avg       0.96      0.92      0.93      1320

[[1162   92]
 [  13   53]]


# hyperparameter tuning for xgboost

In [13]:
final_xg = XGBClassifier(
    subsample=0.8,
    scale_pos_weight=1,
    n_estimators=100,
    max_depth=7, learning_rate=0.3,
    gamma=0.2, colsample_bytree=0.8
)

final_xg.fit(X_train_resampled.to_numpy(), y_train_resampled)


##again for identifying threshold for y_prob


from sklearn.metrics import precision_recall_curve

y_probs = final_xg.predict_proba(X_test.to_numpy())[:, 1]

precision, recall, threshold = precision_recall_curve(y_test, y_probs)

f1_score = 2 * (precision*recall)/(precision+recall)

best_threshold_xg = threshold[f1_score.argmax()]

print(f"Best threshold: {best_threshold_xg}")




#y_pred = final_xg.predict(X_test.to_numpy())

y_pred = (y_probs >= best_threshold_xg).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Best threshold: 0.8878124952316284
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1254
           1       0.55      0.55      0.55        66

    accuracy                           0.95      1320
   macro avg       0.76      0.76      0.76      1320
weighted avg       0.95      0.95      0.95      1320

[[1224   30]
 [  30   36]]


In [16]:
import joblib

joblib.dump(best_threshold_rf, '/content/drive/MyDrive/best_threshold_rf.pkl')
joblib.dump(best_threshold_xg, '/content/drive/MyDrive/best_threshold_xg.pkl')


['/content/drive/MyDrive/best_threshold_xg.pkl']